<a href="https://colab.research.google.com/github/tthogho1/CompareImage/blob/main/BLIP2_to_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# New Section

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/

 checkpoint.zip  'Colab Notebooks'   images   images.zip   test.txt


In [4]:
!pip install metal-sdk
!pip install salesforce-lavis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 28.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━

In [8]:
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess
import os
from metal_sdk.metal import Metal

In [6]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model, vis_processors, _ = load_model_and_preprocess(
   name="blip2_opt", model_type="pretrain_opt2.7b", is_eval=True, device=device
)
vis_processors.keys()

100%|██████████| 1.89G/1.89G [01:29<00:00, 22.7MB/s]


100%|██████████| 409M/409M [00:20<00:00, 21.3MB/s]


dict_keys(['train', 'eval'])

In [18]:
metalIndexId = "64f46eb470de24ef7fefe7c7"
metal = Metal(
  "pk_jEyG+TzAS1czHqStsWJRdEghxfz+rvfpNdBzZ+mY/OU=",   # api-key
  "ci_Klqqk9LpWdWUqA7EO2zlSYrTWLNHyRVGWXUD23Wf0pQ=", # client-id
  metalIndexId,  # index-id
)

folder_path = '/content/drive/MyDrive/images'
file_list = os.listdir(folder_path)
sorted_file_list = sorted(file_list)

In [17]:
#test 1 file

raw_image = Image.open(folder_path + '/' + '1018690405.jpg').convert('RGB')
image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
result = model.generate({"image": image})
print(result)

['an aerial view of a ski slope in the mountains']


In [20]:
def create_metal_doc(filename,text):
  embedded_document = {}
  embedded_document["text"] = text
  embedded_document["index"] = metalIndexId
  metadata = {}
  metadata["id"] = filename.replace(".jpg","")
  print(metadata["id"])
  embedded_document["metadata"] = metadata
  return(embedded_document)

In [21]:
embedded_document_list = []
for file in sorted_file_list:
    print(file)
    raw_image = Image.open(folder_path + '/' + file).convert('RGB')
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
    text = model.generate({"image": image})
    doc = create_metal_doc(file,text)
    embedded_document_list.append(doc)
    if len(embedded_document_list) == 100:
      metal.index_many(embedded_document_list)
      embedded_document_list = []

if len(embedded_document_list) > 0:
  metal.index_many(embedded_document_list)

1000550952.jpg
1000550952
1010004899.jpg
1010004899
1010220186.jpg
1010220186
1010222761.jpg
1010222761
1010433724.jpg
1010433724
1010434248.jpg


KeyboardInterrupt: ignored